## Dependencies

In [18]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

## Loading the Data

In [19]:
folder = '../input/g-research-crypto-forecasting/'

# Train data
base_train = pd.read_csv(folder+'train.csv')

# Assets
asset_mapping = pd.read_csv(folder+'asset_details.csv')
asset_mapping

## Preprocessing Data

In [20]:
# converting the timestamp to minutes
base_train['time'] = pd.to_datetime(base_train['timestamp'], unit='s')
base_train.head(50)

base_train.isnull().sum()

In [21]:
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

In [22]:
btc = base_train.loc[base_train['Asset_ID'] == 1].iloc[-20000:, :]

print(btc.info())

with tf.device('/GPU:0'):
    fig = go.Figure(data=go.Ohlc(x=btc.time,
                open=btc.Open,
                high=btc.High,
                low=btc.Low,
                close=btc.Close))
    fig.show()